In [21]:
import json
import io


In [22]:
json_d = { "displayName": "Conversation Task Example",
  "analysisInput": {
    "conversations": [
      {
        "conversationItems": [
          {
            "text": "Hello, you’re chatting with Rene. How may I help you?",
            "id": "1",
            "role": "Agent",
            "participantId": "Agent_1"
          },
          {
            "text": "Hi, I tried to set up wifi connection for Smart Brew 300 espresso machine, but it didn’t work.",
            "id": "2",
            "role": "Customer",
            "participantId": "Customer_1"
          },
          {
            "text": "I’m sorry to hear that. Let’s see what we can do to fix this issue. Could you please try the following steps for me? First, could you push the wifi connection button, hold for 3 seconds, then let me know if the power light is slowly blinking on and off every second?",
            "id": "3",
            "role": "Agent",
            "participantId": "Agent_1"
          },
          {
            "text": "Yes, I pushed the wifi connection button, and now the power light is slowly blinking.",
            "id": "4",
            "role": "Customer",
            "participantId": "Customer_1"
          },
          {
            "text": "Great. Thank you! Now, please check in your Contoso Coffee app. Does it prompt to ask you to connect with the machine? ",
           "id": "5",
            "role": "Agent",
            "participantId": "Agent_1"
          },
          {
            "text": "No. Nothing happened.",
            "id": "6",
            "role": "Customer",
            "participantId": "Customer_1"
          },
          {
            "text": "I’m very sorry to hear that. Let me see if there’s another way to fix the issue. Please hold on for a minute.",
            "id": "7",
            "role": "Agent",
            "participantId": "Agent_1"
          }
        ],
        "modality": "text",
        "id": "conversation1",
        "language": "en"
      }
    ]
  },
  'tasks': [{'taskName': 'analyze 1',
   'kind': 'ConversationalSummarizationTask',
   'parameters': {'summaryAspects': ['chaptertitle',
     'issue',
     'resolution',
     'narrative']}}]}



In [23]:
import requests
from requests.structures import CaseInsensitiveDict
from configparser import ConfigParser


parser=ConfigParser()
_=parser.read('../config.cfg')
cog_svc_ep = parser.get('openai_api','cog_svc_ep')


url = "{}/language/analyze-conversations/jobs?api-version=2022-10-01-preview".format(cog_svc_ep)

headers = CaseInsensitiveDict()
headers["Content-Type"] = "application/json"
headers["Ocp-Apim-Subscription-Key"] = parser.get('openai_api','cog_svc_key')

data = json_d

resp = requests.post(url, headers=headers, json=data)

print(resp)


<Response [202]>


In [24]:
url = resp.headers['operation-location']

In [25]:
headers = CaseInsensitiveDict()
headers["Content-Type"] = "application/json"
headers["Ocp-Apim-Subscription-Key"] = parser.get('openai_api','cog_svc_key')

data = json_d

resp = requests.get(url, headers=headers)

print(resp)

<Response [200]>


In [26]:
## Wait for Asynchronous call to complete

In [27]:
out= resp.json()

for task in out['tasks']['items'][0]['results']['conversations'][0]['summaries']:
    print("{} = {}".format(task['aspect'],task['text']))

chapterTitle = Setting up WiFi Connection for Smart Brew 300 Espresso Machine
issue = Customer wants to connect their Smart Brew 300 to their Wi-Fi. | The Wi-Fi connection didn't work.
resolution = Recommended customer to check the wifi connection. | Recommended customer to check the contoso coffee app. | Recommended customer to hold on for a minute.
narrative = Agent_1 helps customer to set up wifi connection for Smart Brew 300 espresso machine.


In [28]:
import os
import openai
from configparser import ConfigParser

parser=ConfigParser()
_=parser.read('../config.cfg')
openai.api_type = "azure"
openai.api_base = parser.get('openai_api','api_ep')
openai.api_version = "2022-06-01-preview"
openai.api_key = parser.get('openai_api','api_key')
model =  parser.get('openai_api','api_model')

In [18]:
conversation = json_d['analysisInput']['conversations'][0]

In [29]:
prompt = "Generate detailed call summary notes from the following text\nText:###{}###\n".format(conversation)

response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.5,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print(response['choices'][0]['text'])

Response:

-Customer is having difficulty connecting their Smart Brew 300 espresso machine to wifi
-Agent troubleshoots with customer, asking them to push the wifi connection button and hold for 3 seconds, then checking the power light
-Customer does as asked, and reports that the power light is now slowly blinking
-Agent asks customer to check the Contoso Coffee app, to see if it prompts to connect with the machine
-Customer reports that nothing happened
-Agent troubleshoots further, asking customer to hold on for a minute


In [30]:
prompt = "Generate list of follow up tasks from the following text\nText:###{}###\n".format(conversation)

response = openai.Completion.create( engine=model,  prompt=prompt, temperature=.5,  max_tokens=400,  top_p=0.5,  frequency_penalty=0,  presence_penalty=0,  stop=None)

print('Response:')
print(response['choices'][0]['text'])

Response:

-Try restarting the machine
-If that doesn't work, try unplugging and replugging in the machine
-If the problem persists, contact customer support
